In [1]:
#pip install selenium beautifulsoup4

In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
page=requests.get('https://www.emlakjet.com/kiralik-konut/')

In [4]:
page.content

b'<!DOCTYPE html><html lang="tr"><head><link rel="shortcut icon" type="image/png" href="/static/img/favicon-green.png"/><style>\n    /* roboto-regular - latin_latin-ext */\n    @font-face {\n      font-family: \'Roboto\';\n      font-style: normal;\n      font-display: swap;\n      font-weight: 400;\n      src: url(\'/static/fonts/roboto-regular.eot\'); /* IE9 Compat Modes */\n      src: local(\'Roboto\'), local(\'Roboto-Regular\'),\n        url(\'/static/fonts/roboto-regular.eot?#iefix\') format(\'embedded-opentype\'), /* IE6-IE8 */\n        url(\'/static/fonts/roboto-regular.woff2\') format(\'woff2\'), /* Super Modern Browsers */\n        url(\'/static/fonts/roboto-regular.woff\') format(\'woff\'), /* Modern Browsers */\n        url(\'/static/fonts/roboto-regular.ttf\') format(\'truetype\'), /* Safari, Android, iOS */\n        url(\'/static/fonts/roboto-regular.svg#Roboto\') format(\'svg\'); /* Legacy iOS */\n    }\n    /* roboto-500 - latin_latin-ext */\n    @font-face {\n      font

In [5]:
soup=BeautifulSoup(page.content,'html.parser')

In [6]:
soup.prettify

<bound method Tag.prettify of <!DOCTYPE html>
<html lang="tr"><head><link href="/static/img/favicon-green.png" rel="shortcut icon" type="image/png"/><style>
    /* roboto-regular - latin_latin-ext */
    @font-face {
      font-family: 'Roboto';
      font-style: normal;
      font-display: swap;
      font-weight: 400;
      src: url('/static/fonts/roboto-regular.eot'); /* IE9 Compat Modes */
      src: local('Roboto'), local('Roboto-Regular'),
        url('/static/fonts/roboto-regular.eot?#iefix') format('embedded-opentype'), /* IE6-IE8 */
        url('/static/fonts/roboto-regular.woff2') format('woff2'), /* Super Modern Browsers */
        url('/static/fonts/roboto-regular.woff') format('woff'), /* Modern Browsers */
        url('/static/fonts/roboto-regular.ttf') format('truetype'), /* Safari, Android, iOS */
        url('/static/fonts/roboto-regular.svg#Roboto') format('svg'); /* Legacy iOS */
    }
    /* roboto-500 - latin_latin-ext */
    @font-face {
      font-family: 'Roboto

In [7]:
soup=BeautifulSoup(page.content,'html.parser')

In [8]:
fiyatlar=soup.find_all('p', {'class':'_2C5UCT'})[0].text

In [9]:
fiyatlar

'15.000 TL'

In [10]:
evler =soup.find_all('div', {'class':'manJWF'})[1].text

In [11]:
evler

'BETÜL EMLAKTAN ESKİ HABİBLER MAHALLESİNDE KİRALIK 2+1 DAİRE homeDaireweekend2+1layers1. Kattexture75 m²event31 Ekim25.000 TLİstanbul - Sultangazi - Eski Habipler Mh.'

In [12]:
aciklamalar=soup.find_all('div', {'class':'_1TNSG2'})[0].text

In [13]:
aciklamalar

' ALANYA MAHMUTLAR EŞYALI 3+1'

In [14]:
bilgiler=soup.find_all('div', {'class':'_2UELHn'})[0]

In [15]:
bilgiler

<div class="_2UELHn" data-test-selector="listing-item-property-container-1"><span data-test-selector="listing-item-property-category-1"><i class="material-icons">home</i>Daire</span><span data-test-selector="listing-item-property-room-count-1"><i class="material-icons">weekend</i>3+1</span><span data-test-selector="listing-item-property-floor-number-1"><i class="material-icons">layers</i>Kot 1 (-1). Kat</span><span data-test-selector="listing-item-property-gross-square-1"><i class="material-icons">texture</i>110 m²</span><span data-test-selector="listing-item-property-updated-at-1"><i class="material-icons">event</i><span><span>31 Ekim</span></span></span></div>

In [16]:
import pandas as pd

In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Veri çekilecek URL'nin temel kısmı
base_url = 'https://www.emlakjet.com/kiralik-konut/{}'

# Toplam ilan sayısı ve sayfa numarası
total_listings = 2000
current_page = 1
all_properties = []

while len(all_properties) < total_listings:
    # URL'yi oluştur
    url = base_url.format(current_page)
    print(f'İşleniyor: {url}')
    
    # Sayfayı al
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # İlanları seç
    listing_containers = soup.select('div[data-test-selector^="listing-item-property-container-"]')
    
    # Eğer ilan yoksa döngüyü kır
    if not listing_containers:
        print("İlan bulunamadı, döngüden çıkılıyor...")
        break
    
    # Her bir ilan için bilgileri al
    for container in listing_containers:
        category = container.select_one('span[data-test-selector^="listing-item-property-category-"]')
        room_count = container.select_one('span[data-test-selector^="listing-item-property-room-count-"]')
        floor_number = container.select_one('span[data-test-selector^="listing-item-property-floor-number-"]')
        gross_square = container.select_one('span[data-test-selector^="listing-item-property-gross-square-"]')
        updated_at = container.select_one('span[data-test-selector^="listing-item-property-updated-at-"] span span')

        # Bilgileri kontrol ederek metin değerlerini al veya None olarak belirle
        category = category.text.strip()[4:] if category else None
        room_count = room_count.text.strip()[7:] if room_count else None
        floor_number = floor_number.text.strip()[6:].replace(".", "").replace(" Kat", "").strip() if floor_number else None
        gross_square = gross_square.text.strip()[7:].replace("m²", "").strip() if gross_square else None
        updated_at = updated_at.text.strip() if updated_at else None

        # Tüm verileri bir dictionary içinde saklayarak listeye ekleyin
        all_properties.append({
            'Category': category,
            'Room Count': room_count,
            'Floor Number': floor_number,
            'Gross Square': float(gross_square) if gross_square else None,
            'Updated At': updated_at
        })

    # Sayfa sayısını bir arttır
    current_page += 1

    # Her sayfa arasında bekleme süresi (isteğe bağlı)
    time.sleep(1)



İşleniyor: https://www.emlakjet.com/kiralik-konut/1
İşleniyor: https://www.emlakjet.com/kiralik-konut/2
İşleniyor: https://www.emlakjet.com/kiralik-konut/3
İşleniyor: https://www.emlakjet.com/kiralik-konut/4
İşleniyor: https://www.emlakjet.com/kiralik-konut/5
İşleniyor: https://www.emlakjet.com/kiralik-konut/6
İşleniyor: https://www.emlakjet.com/kiralik-konut/7
İşleniyor: https://www.emlakjet.com/kiralik-konut/8
İşleniyor: https://www.emlakjet.com/kiralik-konut/9
İşleniyor: https://www.emlakjet.com/kiralik-konut/10
İşleniyor: https://www.emlakjet.com/kiralik-konut/11
İşleniyor: https://www.emlakjet.com/kiralik-konut/12
İşleniyor: https://www.emlakjet.com/kiralik-konut/13
İşleniyor: https://www.emlakjet.com/kiralik-konut/14
İşleniyor: https://www.emlakjet.com/kiralik-konut/15
İşleniyor: https://www.emlakjet.com/kiralik-konut/16
İşleniyor: https://www.emlakjet.com/kiralik-konut/17
İşleniyor: https://www.emlakjet.com/kiralik-konut/18
İşleniyor: https://www.emlakjet.com/kiralik-konut/19
İş

In [18]:
# Listeyi DataFrame'e dönüştür
df = pd.DataFrame(all_properties)

# DataFrame'i yazdır
print(df)

# İlan sayısını yazdır
print(f'Toplam ilan sayısı: {len(df)}')

     Category Room Count  Floor Number  Gross Square Updated At
0       Daire        3+1  Yüksek giriş         110.0    31 Ekim
1       Daire        2+1             1          75.0    31 Ekim
2       Daire        2+1             1          90.0    31 Ekim
3       Daire        2+1             4          95.0    31 Ekim
4       Daire        1+1             1          55.0    31 Ekim
...       ...        ...           ...           ...        ...
2005    Daire        3+1             1         150.0    28 Ekim
2006    Daire        3+1             3         125.0    28 Ekim
2007    Daire        1+1             5          65.0    28 Ekim
2008    Daire        3+1             1         135.0    28 Ekim
2009    Daire        3+1             2         140.0    28 Ekim

[2010 rows x 5 columns]
Toplam ilan sayısı: 2010


In [19]:
df.shape

(2010, 5)

In [20]:
#price df oluşturuyoruz
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Veri çekilecek URL'nin temel kısmı
base_url = 'https://www.emlakjet.com/kiralik-konut/{}'

# Toplam ilan sayısı ve sayfa numarası
total_listings = 2000
current_page = 1
all_prices = []

while len(all_prices) < total_listings:
    # URL'yi oluştur
    url = base_url.format(current_page)
    print(f'İşleniyor: {url}')
    
    # Sayfayı al
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Fiyatları seç
    price_containers = soup.select('p[data-test-selector^="listing-item-price-"]')
    
    # Eğer fiyat yoksa döngüyü kır
    if not price_containers:
        print("Fiyat bulunamadı, döngüden çıkılıyor...")
        break
    
    # Fiyat bilgilerini al
    for price in price_containers:
        # Fiyatı düzenle
        price_value = price.text.strip().replace("TL", "").replace(".", "").replace(",", "").strip()
        
        # Fiyatı integer olarak kaydet
        all_prices.append(price_value)
        
        # Eğer 2000 veriye ulaştıysak döngüyü kır
        if len(all_prices) >= total_listings:
            break

    # Sayfa sayısını bir arttır
    current_page += 1


İşleniyor: https://www.emlakjet.com/kiralik-konut/1
İşleniyor: https://www.emlakjet.com/kiralik-konut/2
İşleniyor: https://www.emlakjet.com/kiralik-konut/3
İşleniyor: https://www.emlakjet.com/kiralik-konut/4
İşleniyor: https://www.emlakjet.com/kiralik-konut/5
İşleniyor: https://www.emlakjet.com/kiralik-konut/6
İşleniyor: https://www.emlakjet.com/kiralik-konut/7
İşleniyor: https://www.emlakjet.com/kiralik-konut/8
İşleniyor: https://www.emlakjet.com/kiralik-konut/9
İşleniyor: https://www.emlakjet.com/kiralik-konut/10
İşleniyor: https://www.emlakjet.com/kiralik-konut/11
İşleniyor: https://www.emlakjet.com/kiralik-konut/12
İşleniyor: https://www.emlakjet.com/kiralik-konut/13
İşleniyor: https://www.emlakjet.com/kiralik-konut/14
İşleniyor: https://www.emlakjet.com/kiralik-konut/15
İşleniyor: https://www.emlakjet.com/kiralik-konut/16
İşleniyor: https://www.emlakjet.com/kiralik-konut/17
İşleniyor: https://www.emlakjet.com/kiralik-konut/18
İşleniyor: https://www.emlakjet.com/kiralik-konut/19
İş

In [21]:
# Listeyi DataFrame'e dönüştür
prices_df = pd.DataFrame(all_prices, columns=['Price'])

# Toplam ilan sayısını hesapla
total_collected_listings = len(prices_df)

# DataFrame'i yazdır
print(prices_df)

# Toplam fiyat sayısını yazdır
print(f'Toplam fiyat sayısı: {total_collected_listings}')

      Price
0     25000
1     28000
2     19000
3     16000
4     13500
...     ...
1995  15000
1996  16000
1997  20000
1998   9000
1999  12999

[2000 rows x 1 columns]
Toplam fiyat sayısı: 2000


In [22]:
prices_df.head()

,Price
0,25000
1,28000
2,19000
3,16000
4,13500


In [23]:
prices_df.shape

(2000, 1)

In [24]:
df.head()

,Category,Room Count,Floor Number,Gross Square,Updated At
0,Daire,3+1,Yüksek giriş,110.0,31 Ekim
1,Daire,2+1,1,75.0,31 Ekim
2,Daire,2+1,1,90.0,31 Ekim
3,Daire,2+1,4,95.0,31 Ekim
4,Daire,1+1,1,55.0,31 Ekim


In [25]:
df.shape

(2010, 5)

In [26]:
df["price"]= prices_df

In [27]:
df.head()

,Category,Room Count,Floor Number,Gross Square,Updated At,price
0,Daire,3+1,Yüksek giriş,110.0,31 Ekim,25000
1,Daire,2+1,1,75.0,31 Ekim,28000
2,Daire,2+1,1,90.0,31 Ekim,19000
3,Daire,2+1,4,95.0,31 Ekim,16000
4,Daire,1+1,1,55.0,31 Ekim,13500


In [28]:
df.isnull().sum()

Category         0
Room Count       0
Floor Number    38
Gross Square     0
Updated At       0
price           10
dtype: int64

In [29]:
df = df.dropna(subset=['Floor Number', "price"])

In [30]:
df.head()

,Category,Room Count,Floor Number,Gross Square,Updated At,price
0,Daire,3+1,Yüksek giriş,110.0,31 Ekim,25000
1,Daire,2+1,1,75.0,31 Ekim,28000
2,Daire,2+1,1,90.0,31 Ekim,19000
3,Daire,2+1,4,95.0,31 Ekim,16000
4,Daire,1+1,1,55.0,31 Ekim,13500


In [31]:
df.shape

(1962, 6)

In [32]:
df.isnull().sum()

Category        0
Room Count      0
Floor Number    0
Gross Square    0
Updated At      0
price           0
dtype: int64

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1962 entries, 0 to 1999
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Category      1962 non-null   object 
 1   Room Count    1962 non-null   object 
 2   Floor Number  1962 non-null   object 
 3   Gross Square  1962 non-null   float64
 4   Updated At    1962 non-null   object 
 5   price         1962 non-null   object 
dtypes: float64(1), object(5)
memory usage: 107.3+ KB


In [34]:
df["price"]=df["price"].replace({
    "450 GBP": "20000",
    "650 GBP" :"28600",
    "550 GBP":"24200"
})

In [35]:
df[df['price'].str.contains("GBP")]

,Category,Room Count,Floor Number,Gross Square,Updated At,price


In [36]:
df["price"]=df["price"].astype(float)

In [37]:
df["Updated At"].value_counts()

Updated At
31 Ekim       820
30 Ekim       679
29 Ekim       246
28 Ekim       133
26 Ekim        16
27 Ekim        13
25 Ekim         8
19 Ekim         6
22 Ekim         4
24 Ekim         3
23 Ekim         3
18 Ekim         3
17 Ekim         3
10 Ekim         2
20 Eylül        2
9 Ekim          2
20 Ekim         2
15 Ekim         1
1 Ekim          1
28 Eylül        1
7 Ekim          1
24 Eylül        1
21 Ekim         1
26 Ağustos      1
26 Eylül        1
15 Eylül        1
12 Ekim         1
23 Eylül        1
13 Ekim         1
25 Eylül        1
11 Ekim         1
30 Eylül        1
4 Ekim          1
10 Eylül        1
Name: count, dtype: int64

In [38]:
df[['day', 'month']] = df['Updated At'].str.split(' ', expand=True)

In [39]:
df.head()

,Category,Room Count,Floor Number,Gross Square,Updated At,price,day,month
0,Daire,3+1,Yüksek giriş,110.0,31 Ekim,25000.0,31,Ekim
1,Daire,2+1,1,75.0,31 Ekim,28000.0,31,Ekim
2,Daire,2+1,1,90.0,31 Ekim,19000.0,31,Ekim
3,Daire,2+1,4,95.0,31 Ekim,16000.0,31,Ekim
4,Daire,1+1,1,55.0,31 Ekim,13500.0,31,Ekim


In [40]:
df["month"]=df["month"].replace({
    "Ekim": 10,
    "Eylül":9,
    "Ağustos":8
})

C:\Users\karsa\AppData\Local\Temp\ipykernel_5608\900764802.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["month"]=df["month"].replace({


In [41]:
df["month"].value_counts()

month
10    1951
9       10
8        1
Name: count, dtype: int64

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1962 entries, 0 to 1999
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Category      1962 non-null   object 
 1   Room Count    1962 non-null   object 
 2   Floor Number  1962 non-null   object 
 3   Gross Square  1962 non-null   float64
 4   Updated At    1962 non-null   object 
 5   price         1962 non-null   float64
 6   day           1962 non-null   object 
 7   month         1962 non-null   int64  
dtypes: float64(2), int64(1), object(5)
memory usage: 138.0+ KB


In [43]:
df["day"] = df["day"].astype(float)

In [44]:
df["dayago"]=(10 - df["month"])*30 + df["day"]

In [45]:
df["dayago"].unique()

array([31., 30., 29., 28., 27., 26., 25., 24., 23., 22., 21., 20., 19.,
       18., 17., 15., 10.,  9.,  4., 60., 11., 55., 13., 53., 12., 45.,
       56., 86., 50., 54.,  7., 58.,  1., 40.])

In [46]:
df["dayago"] = 100- df["dayago"]

In [47]:
df.corr(numeric_only=True)["price"]

Gross Square   -0.018124
price           1.000000
day            -0.025521
month          -0.041051
dayago         -0.014850
Name: price, dtype: float64

In [48]:
df[['Room1', 'Room2']] = df['Room Count'].str.split('+', expand=True)

In [49]:
df['Room1'].unique()

array(['3', '2', '1', '3.5', '4', '1 Oda', '5', '9', 'Stüdyo', '2.5',
       '4.5', '6'], dtype=object)

In [50]:
df["Room1"]=df["Room1"].replace({
    "1 Oda": 1,
    "Stüdyo":1
})

In [51]:
df['Room2'].unique()

array(['1', None, '0', ' Oda', '2'], dtype=object)

In [52]:
df["Room2"]=df["Room2"].replace({
    " Oda": 1
})

In [53]:
df['Room1'] = df['Room1'].astype(float)
df['Room2'] = df['Room2'].astype(float)

In [54]:
df.isnull().sum()

Category         0
Room Count       0
Floor Number     0
Gross Square     0
Updated At       0
price            0
day              0
month            0
dayago           0
Room1            0
Room2           31
dtype: int64

In [55]:
df['Room2'] =df['Room2'].fillna(0)

In [56]:
df.isnull().sum()

Category        0
Room Count      0
Floor Number    0
Gross Square    0
Updated At      0
price           0
day             0
month           0
dayago          0
Room1           0
Room2           0
dtype: int64

In [57]:
df["Category"].value_counts()

Category
Daire           1941
Residence         18
Yazlık             2
Yalı Dairesi       1
Name: count, dtype: int64

In [58]:
df["Floor Number"].value_counts()

Floor Number
2                    375
1                    344
3                    338
4                    219
5                    122
Yüksek giriş         121
Düz Giriş (Zemin)     95
Bahçe katı            68
7                     47
6                     40
Kot 1 (-1)            31
8                     24
9                     17
Çatı Dubleks          17
Kot 2 (-2)            14
Çatı katı             14
11                     9
12                     8
Kot 3 (-3)             8
13                     6
Bodrum                 6
14                     5
10                     5
Villa tipi             4
Bahçe dublex           4
18                     4
16                     3
Müstakil               3
17                     2
26                     2
15                     2
38                     1
Kot 4 (-4)             1
19                     1
31                     1
25                     1
Name: count, dtype: int64

In [59]:
df["Floor Number"] = df["Floor Number"].replace({
    "Yüksek giriş": 0,
    "Düz Giriş (Zemin)": 0,
    "Bahçe katı": 0,
    "Villa tipi": 0,
    "Bahçe dublex": 0,
    "Müstakil": 0,
    "Çatı katı": 4,
    "Çatı Dubleks": 4,
    "Kot 2 (-2)": 0,
    "Kot 1 (-1)": 0,
    "Bodrum": 0,
    "Kot 3 (-3)": 0,
    "Kot 4 (-4)": -0
})


In [60]:
df["Floor Number"].value_counts()

Floor Number
2     375
0     355
1     344
3     338
4     219
5     122
7      47
6      40
4      31
8      24
9      17
11      9
12      8
13      6
14      5
10      5
18      4
16      3
17      2
26      2
15      2
38      1
19      1
31      1
25      1
Name: count, dtype: int64

In [61]:
df["Floor Number"]=df["Floor Number"].astype(float)

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1962 entries, 0 to 1999
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Category      1962 non-null   object 
 1   Room Count    1962 non-null   object 
 2   Floor Number  1962 non-null   float64
 3   Gross Square  1962 non-null   float64
 4   Updated At    1962 non-null   object 
 5   price         1962 non-null   float64
 6   day           1962 non-null   float64
 7   month         1962 non-null   int64  
 8   dayago        1962 non-null   float64
 9   Room1         1962 non-null   float64
 10  Room2         1962 non-null   float64
dtypes: float64(7), int64(1), object(3)
memory usage: 183.9+ KB


In [63]:
df=df[["Category", "Floor Number","Gross Square", "dayago" ,"Room1","Room2","price"]]

In [64]:
df=pd.get_dummies(df,drop_first=True)

In [65]:
x=df.drop(['price'], axis=1)

In [66]:
y = df[['price']]

In [67]:
#regression

In [68]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [69]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error

def algo_test(x,y):
  #Bütün modelleri tanımlıyorum
  L=LinearRegression()
  R=Ridge()
  Lass=Lasso()
  E=ElasticNet()
  ETR=ExtraTreeRegressor()
  GBR=GradientBoostingRegressor()
  kn=KNeighborsRegressor()
  dt=DecisionTreeRegressor()
  xgb=XGBRegressor()
  rf = RandomForestRegressor()

  algos=[L,R,Lass,E,ETR,GBR,kn,dt,xgb,rf]
  algo_names=['Linear','Ridge','Lasso','ElasticNet','Extra Tree','Gradient Boosting','KNeighborsRegressor','Decision Tree Regressor','XGBRegressor', 'RandomForestRegressor']

  x_train, x_test, y_train, y_test=train_test_split(x,y,test_size =.2,random_state=42)

  r_squared= []
  rmse= []
  mae= []

  #Hata ve doğruluk oranlarım bir tablo haline getirmek için bir dataframe oluşturuyorum
  result=pd.DataFrame(columns=['R_Squared','RMSE','MAE'],index=algo_names)

  for algo in algos:
    p=algo.fit(x_train,y_train).predict(x_test)
    r_squared.append(r2_score(y_test,p))
    rmse.append(mean_squared_error(y_test,p) **.5)
    mae. append(mean_absolute_error(y_test,p))

  #result adlı tabloya doğruluk ve hata oranlarum yerleştiriyorum
  result.R_Squared=r_squared
  result.RMSE=rmse
  result.MAE=mae

  #oluşturduğum result tablosunu doğruluk oranına (r2_score) göre sıralayıp dönüyor
  rtable=result.sort_values('R_Squared',ascending=False)
  return rtable

In [70]:
algo_test(x,y)

C:\Users\karsa\anaconda3\envs\env\lib\site-packages\sklearn\ensemble\_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?
C:\Users\karsa\anaconda3\envs\env\lib\site-packages\sklearn\base.py:1351: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


,R_Squared,RMSE,MAE
Ridge,-0.009552,1.556278e+06,169986.047562
Linear,-0.009554,1.556280e+06,170011.532844
Lasso,-0.009554,1.556280e+06,170009.392754
ElasticNet,-0.010246,1.556813e+06,169592.812121
Gradient Boosting,-0.017293,1.562234e+06,178163.447800
KNeighborsRegressor,-0.051127,1.588001e+06,179437.361323
RandomForestRegressor,-0.409353,1.838793e+06,228701.267143
Decision Tree Regressor,-0.701112,2.020177e+06,250114.993806
XGBRegressor,-0.837622,2.099670e+06,279537.546012
Extra Tree,-1.044491,2.214703e+06,294732.912320


In [71]:
#DNN

In [72]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [73]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [74]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(x_train.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')
history = model.fit(x_train, y_train, epochs=10, batch_size=16, validation_split=0.2)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [ ]:
# Modeli değerlendirme
y_pred_nn = model.predict(X_test).flatten()
rmse_nn = mean_squared_error(y_test, y_pred_nn, squared=False)
print(f'Neural Network RMSE: {rmse_nn:.2f}')

In [ ]:
Sonuç, web scraping yapılıp dataset oluşturulmuştur. Feature engineering yapılarak data düzeltilmiştir. Hata nerde bilmiyorum ama r score ilginç bide değer geldi.

In [75]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Veriyi yükleme ve hazırlama (df veri çerçevenizin olduğunu varsayıyorum)
# 'price' hedef değişken, diğer sütunlar ise öznitelikler olarak kullanılacak.
X = df.drop(columns=["price"])
y = df["price"]

# Veriyi eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Veriyi ölçeklendirme
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Modeli oluşturma
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='linear')  # Fiyat tahmini için lineer aktivasyon
])

# Modeli derleme
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Modeli eğitme
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Test setinde modeli değerlendirme
test_loss, test_mae = model.evaluate(X_test, y_test)
print("Test MAE:", test_mae)

# Fiyat tahmini yapmak için kullanabilirsiniz
predictions = model.predict(X_test)


Epoch 1/50
40/40 [==============================] - 1s 10ms/step - loss: 293853233152.0000 - mae: 44219.0664 - val_loss: 31413143552.0000 - val_mae: 31781.7930
Epoch 2/50
40/40 [==============================] - 0s 2ms/step - loss: 293853069312.0000 - mae: 44217.9727 - val_loss: 31412981760.0000 - val_mae: 31780.2207
Epoch 3/50
40/40 [==============================] - 0s 2ms/step - loss: 293852938240.0000 - mae: 44215.6055 - val_loss: 31412547584.0000 - val_mae: 31776.5215
Epoch 4/50
40/40 [==============================] - 0s 2ms/step - loss: 293852446720.0000 - mae: 44209.9141 - val_loss: 31411662848.0000 - val_mae: 31768.7891
Epoch 5/50
40/40 [==============================] - 0s 2ms/step - loss: 293851693056.0000 - mae: 44199.6328 - val_loss: 31410124800.0000 - val_mae: 31754.9648
Epoch 6/50
40/40 [==============================] - 0s 2ms/step - loss: 293850251264.0000 - mae: 44182.1367 - val_loss: 31407366144.0000 - val_mae: 31730.3398
Epoch 7/50
40/40 [===========================